# NOTES
Right now only takes close price. Update to include all data in the sequence.

After 3 epochs it's loss is basically non-existant. Use an early stop

Learn how to use GPU 

In [117]:
import pandas as pd
import numpy as np
from sklearn.model_selection import TimeSeriesSplit
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import TensorDataset, DataLoader
import torchmetrics

In [118]:
DATA_PATH = r'C:\Users\connor\PycharmProjects\trading\data\analytics\analytics_voo.csv'
df = pd.read_csv(DATA_PATH, low_memory=False)

In [119]:
df.columns

Index(['date', 'open', 'high', 'low', 'close', 'volume', 'join_date', 'Id',
       'Date', 'DateLongDescription', 'DateShortDescription', 'DayLongName',
       'DayShortName', 'MonthLongName', 'MonthShortName', 'CalendarDay',
       'CalendarWeek', 'CalendarWeekStartDateId', 'CalendarWeekEndDateId',
       'CalendarDayInWeek', 'CalendarMonth', 'CalendarMonthStartDateId',
       'CalendarMonthEndDateId', 'CalendarNumberOfDaysInMonth',
       'CalendarDayInMonth', 'CalendarQuarter', 'CalendarQuarterStartDateId',
       'CalendarQuarterEndDateId', 'CalendarQuarterStartDate',
       'CalendarNumberOfDaysInQuarter', 'CalendarDayInQuarter', 'CalendarYear',
       'CalendarYearEndDateId', 'CalendarYearStartDate',
       'CalendarNumberOfDaysInYear', 'month_join_key', 'year_join_key',
       'seven_day_ema', 'CPALTT01USM657N', 'DFF', 'EXPINF10YR', 'GDPC1',
       'RSXFS', 'T10YFF', 'UNRATE', 'macd', 'macd_signal', 'macd_hist',
       'daily_obv', 'target'],
      dtype='object')

In [120]:
non_target_columns = ['open', 'high', 'low', 'volume', 'CPALTT01USM657N', 'DFF', 'EXPINF10YR', 'GDPC1', 'RSXFS', 'T10YFF', 'UNRATE', 'macd', 'macd_signal', 'macd_hist', 'daily_obv', 'seven_day_ema', 'close']

df_cols = df[non_target_columns]

df_target_train = df[['date','close']].where(df.date <= '2020-01-01')
df_target_train.dropna(inplace=True)
df_target_test = df[['date','close']].where(df.date > '2020-01-01')
df_target_test.dropna(inplace=True)

In [121]:
num_features = df_cols.shape[1]

In [122]:
# creating sequences
# sequence right now only accounts for close price. Could probably include the non_target_columns as values here too
def create_sequences(df, seq_length, num_rows=len(df)):
    xs, ys = [], []
    for i in range(num_rows - seq_length):
        x = df.iloc[i:(i + seq_length), 1]
        y = df.iloc[i + seq_length, 1]
        xs.append(x)
        ys.append(y)
    return np.array(xs), np.array(ys)

In [123]:
# create training set
sequence_len = 10
num_rows = 100000
X_train, y_train = create_sequences(df_target_train, sequence_len, num_rows)
X_test, y_test = create_sequences(df_target_test, sequence_len, num_rows)

In [124]:
print('x train', X_train)
print('x test', X_test)

x train [[ 83.967  83.889  83.92  ...  84.169  84.184  84.278]
 [ 83.889  83.92   83.923 ...  84.184  84.278  84.169]
 [ 83.92   83.923  83.998 ...  84.278  84.169  84.241]
 ...
 [104.337 104.365 104.349 ... 104.354 104.347 104.397]
 [104.365 104.349 104.365 ... 104.347 104.397 104.397]
 [104.349 104.365 104.351 ... 104.397 104.397 104.413]]
x test [[278.923 279.055 279.074 ... 279.13  279.177 279.233]
 [279.055 279.074 279.13  ... 279.177 279.233 279.205]
 [279.074 279.13  279.13  ... 279.233 279.205 279.186]
 ...
 [301.377 301.271 301.337 ... 301.242 301.214 301.271]
 [301.271 301.337 301.052 ... 301.214 301.271 301.199]
 [301.337 301.052 301.152 ... 301.271 301.199 300.805]]


In [125]:
print(X_train.shape, y_train.shape)

(99990, 10) (99990,)


In [126]:
# # right now the model is only going to be trained on the last split. Keep it this way for now
# n_splits = 5
# tscv  = TimeSeriesSplit(n_splits)
# 
# X_train = pd.DataFrame()
# y_train = pd.DataFrame()
# X_test = pd.DataFrame()
# y_test = pd.DataFrame()
# 
# for i, (train_index, test_index) in enumerate(tscv.split(df)): 
#     # split data for each time series partition 
#     y_train = df[target_column].iloc[train_index]
#     X_train = df[non_target_columns].iloc[train_index]
#     y_test = df[target_column].iloc[test_index]
#     X_test = df[non_target_columns].iloc[test_index]

In [127]:
# convert df into objects Torch can read
torch_X_train  = torch.from_numpy(X_train).float()
torch_y_train = torch.from_numpy(y_train).float()
torch_X_test = torch.from_numpy(X_test).float()
torch_y_test = torch.from_numpy(y_test).float()

# create test and train sets
train_data_set = TensorDataset(torch_X_train, torch_y_train)
test_data_set = TensorDataset(torch_X_test, torch_y_test)

# confirm it works
sample = train_data_set[0]
input_sample, label_sample = sample
print('input sample:', input_sample)
print('label sample:', label_sample)

input sample: tensor([83.9670, 83.8890, 83.9200, 83.9230, 83.9980, 84.2000, 84.3090, 84.1690,
        84.1840, 84.2780])
label sample: tensor(84.1690)


In [128]:
print(input_sample.shape)

torch.Size([10])


In [129]:
batch_size = 10
shuffle = True
hidden_size = 5
num_layers = 2

# create dataloader
train_dataloader = DataLoader(train_data_set, batch_size=batch_size, shuffle=shuffle)
test_dataloader = DataLoader(test_data_set, batch_size=batch_size, shuffle=shuffle)
# test loader
# x, y = next(iter(train_dataloader))
# 
# print('x', x, 'y', y)

In [130]:
# LTSM
class Net(nn.Module):
    def __init__(self, input_size):
        super(Net, self).__init__() #super makes all the methods available in nn.Module available for the new class Net
        self.lstm = nn.LSTM(
            input_size=input_size,
            hidden_size=hidden_size,
            num_layers=num_layers,
            batch_first=True
        )
        self.fc = nn.Linear(in_features=hidden_size, out_features=1)
        
    def forward(self, x):
        h0 = torch.zeros(num_layers, x.size(0), hidden_size)
        c0 = torch.zeros(num_layers, x.size(0), hidden_size)
        out, _ = self.lstm(x, (h0, c0))
        out = self.fc(out[:, -1, :])
        return out

In [131]:
learning_rate = 0.001
num_epochs = 50
num_features = 1 # this right now is just the close date. 

net = Net(input_size=num_features)
#criterion = nn.BCELoss(reduction='sum') #you'll need to use the binary close price higher for this
criterion = nn.MSELoss()
optimizer = optim.Adam(
    net.parameters(), lr=learning_rate
)

for epoch in range(num_epochs):
    for seqs, labels in train_dataloader:
        seqs = seqs.view(batch_size, sequence_len, num_features)
        outputs = net(seqs).squeeze()
        loss = criterion(outputs, labels)
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
    print(f"Epoch {epoch+1}, Loss: {loss.item()}")
    # early break if loss isn't changing beyond the learning rate
    if loss.item() < .1:
        break

Epoch 1, Loss: 1360.403076171875
Epoch 2, Loss: 24.176294326782227
Epoch 3, Loss: 34.915611267089844
Epoch 4, Loss: 0.16527804732322693
Epoch 5, Loss: 0.015886306762695312


In [132]:
# Define MSE metric
mse = torchmetrics.regression.MeanSquaredError()

net.eval()
with torch.no_grad():
    for seqs, labels in test_dataloader:
        seqs = seqs.view(batch_size, sequence_len, num_features)
        # Pass seqs to net and squeeze the result
        outputs = net(seqs).squeeze()
        mse(outputs, labels)

# Compute final metric value
test_mse = mse.compute()
print(f"Test MSE: {test_mse}")
print(f"Test RMSE: {test_mse**.5}")

Test MSE: 25950.716796875
Test RMSE: 161.09225463867188


In [133]:
## used for binary classifcation
# f1 = torchmetrics.F1Score(num_classes=2)
# 
# net.eval()
# with torch.no_grad():
#     for seqs, labels in test_dataloader:
#         seqs = seqs.view(batch_size, num_features, 1)
#         outputs = net(seqs).squeeze()
#         f1.compute()
#         
# print(f"Test F1 score: {f1.compute()}")

# acc = torchmetrics.Accuracy(task="binary")
# 
# net.eval()
# with torch.no_grad():
#     for seqs, labels in test_dataloader:
#         seqs = seqs.view(batch_size, sequence_len, num_features)
#         outputs = net(seqs).squeeze()
#         acc.compute()
# 
# print(f"Test accuracy score: {acc.compute()}")
